<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 2*

# Document Representations: Bag-Of-Words

In [19]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# For Web Scraping
!pip install lxml
!pip install bs4
import requests
import bs4
from bs4 import BeautifulSoup

import time

pd.options.mode.chained_assignment = None

from sklearn.neighbors import NearestNeighbors

Requirement already up-to-date: nltk in /anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages (3.4.4)


[nltk_data] Downloading package punkt to /Users/azel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/azel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [2]:
# I'm going to use this as a template. 
#When I get the scraped data working they way I 
df = pd.read_csv('job_listings.csv')
df.head()

,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


## 2) Use Spacy to tokenize / clean the listings 

In [3]:
##### Your Code Here #####

def cleaner():
    
    cleaned_entry = []
    
    for i in np.arange(len(df)):
        
        sample = df['description'][i]
        soup = BeautifulSoup(sample, "html.parser")
        
        sample = (re.sub(r'[^a-zA-Z ^0-9]', '', 
                         soup.text)[1:])
        
        replace_list = ['xe2x80x99','xe2x80xa6n','xe2x80x99',
                        'xe2x80x93']
        
        for item in replace_list :
            sample = sample.replace(item, ' ')
            
        cleaned_entry.append(sample)

    return cleaned_entry
        

In [4]:
cleaned = cleaner()
df['cleaned'] = cleaned

tokens = []

for i in np.arange(len(df)):

    token = nltk.word_tokenize(df['cleaned'][i])
    token = [x.lower() for x in token]
    tokens.append(token)
    
df['tokens'] = tokens

In [5]:
flat_list = [item for sublist in tokens for item in sublist]

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [7]:
##### Your Code Here #####
# create the transform
vectorizer = CountVectorizer(stop_words='english',
                             lowercase=False)

# tokenize and build vocab
vectorizer.fit(cleaned)

# Create a Vocabulary
# The vocabulary establishes all of the possible words that we might use.
vectorizer.vocabulary_

# The vocabulary dictionary does not represent the counts of words!!
dtm = vectorizer.transform(cleaned)

In [8]:
dtm = pd.DataFrame(dtm.todense(), columns = vectorizer.get_feature_names())
# summarize encoded vector
dtm.head()

,02,02115nJob,030nnMicrosoft,031819,032519,041819,06366,10,100,1000,...,younthe,youre,yourselfnnWe,yoursxe2x80x94is,youve,yrs,yrsnnPosition,zero,zonesnAbility,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 4) Visualize the most common word counts

In [16]:
##### Your Code Here #####
dtm.max().sort_values(ascending=False).head(20)

States       48
United       48
Americas     47
data         40
Data         20
ability      19
Ericsson     18
business     18
team         18
learning     15
Science      14
marketing    14
You          14
Universal    14
Microsoft    14
Autodesk     13
senior       13
science      13
solutions    12
product      12
dtype: int64

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [17]:
##### Your Code Here #####
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', 
                        max_features = 5000, 
                        min_df=5)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(cleaned)

#dtm = tfidf.transofmr(new_data)

# Print word counts

# Get feature names to use as dataframe column headers

# View Feature Matrix as DataFrame
docs = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
docs.head()

,10,100,100000,12,15,20,200,2013,2015,2018,...,xe2x80x9cbig,year,years,yes,york,youll,younnabout,younyou,youre,youve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.059669,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.14617,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 # 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [36]:
##### Your Code Here #####
ideal_job = 'This job has nothing to do with natural language processing. It moslty involves complex financial models and has a cool name'

cleaned.append(ideal_job)

tfidf = TfidfVectorizer(stop_words='english', 
                        max_features = 5000, 
                        min_df=5)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(cleaned)

In [41]:
# Fit on TF-IDF Vectors
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

nn.fit(dtm.todense())

nn.kneighbors(dtm.todense()[-1])

(array([[0.        , 1.16037629, 1.20488201, 1.20666098, 1.21373859]]),
 array([[425, 201, 307, 383, 388]]))

In [47]:
len(dtm.todense())

426

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.